# Burn Area Processing
The purpose of this notebook is to find areas of Montana that have been burned by wildfires over time, and make informed decisions based on the findings.

Though NASA offers a product called [FIRMS](https://firms.modaps.eosdis.nasa.gov) (Fire Information for Resource Managment System) which provides current and historic informations on active wildfires, you should not use this data to find land cover that has been burned.

From the FIRMS [FAQ Page](https://www.earthdata.nasa.gov/faq/firms-faq):

> It is not recommended to use active fire locations to estimate burned area due to spatial and temporal sampling issues. Determining this to an acceptable degree of accuracy is generally not possible due to nontrivial spatial and temporal sampling issues. For some applications, however, acceptable accuracy can be achieved, although the effective area burned per fire pixel is not simply a constant, but rather varies with respect to several different vegetation and fire-related variables. See [Giglio et al. (2006)](https://acp.copernicus.org/articles/6/957/2006/) for more information.

Instead, we will use Dr. Giglio's [MODIS Burn Area Product](https://modis-fire.umd.edu/ba.html) to calculate wildfire damage. 

The product we are interested in is `MCD64 Burned Area Collection 6`. The user guide is located [here](https://modis-fire.umd.edu/files/MODIS_C6_BA_User_Guide_1.3.pdf), but the process will be walked through in this example. These products are hosted on an SFTP server hosted by the University of Maryland.

### If you prefer an app-based SFTP client, follow these steps:

* Open the SFTP program. Enter server address `fuoco.geog.umd.edu` , username `fire` , and password `burnt` . Log in.

* Navigate the directory structure through `data` > `MODIS` > `C6` > `MCD64A1` > `SHP` .

* Enter folder `Win03` , which represents the continental United States. Refer to the user guide if your area of interest resides elsewhere.

* The resulting folder structure returns the years the product has produced. Make a folder within this project folder `ESJ-Montana` called `burnarea` and navigate to it in the opposite column of your SFTP client.

* Drag-and-drop all relevant years into this `burnarea` folder. Wait for the download to complete.

* Skip the next code block and do not run it, as it will just repeat the steps above.

### If you wish to automatically download all relevant files, run the below cell block and enter your desired values when prompted:




In [1]:
# Only run this if you haven't manually downloaded burn area data!

import pysftp
from pathlib import Path
import os

burn_area_folder = "burnarea"
Path(burn_area_folder).mkdir(parents=True, exist_ok=True)

# SFTP server details
sftp_host = "fuoco.geog.umd.edu"
username = "fire"
password = "burnt"

geog_region = "Win03" # Change this for a different geographical area: refer to user guide
remote_directory = "data/MODIS/C6/MCD64A1/SHP/" + geog_region
local_directory = burn_area_folder

# Establish an SFTP connection and list year directories
with pysftp.Connection(host=sftp_host, username=username, password=password) as sftp:
    with sftp.cd(remote_directory):
        all_directories = sftp.listdir()  # List all years

# Filter for year directories
year_directories = [dir for dir in all_directories if dir.isdigit() and 2000 <= int(dir) <= 9999]

# Prompt for start and end year
print(f"Available years: {year_directories}")
start_year = int(input("Enter the beginning of the range to download: "))
end_year = int(input("Enter the end of the range to download: "))

# Validate user input
if str(start_year) not in year_directories or str(end_year) not in year_directories or start_year > end_year:
    print("Invalid year range selected.")

else:
    # Establish another SFTP connection to download files
    with pysftp.Connection(host=sftp_host, username=username, password=password) as sftp:
        for year in range(start_year, end_year + 1):
            year_dir = os.path.join(remote_directory, str(year))
            local_year_dir = os.path.join(local_directory, str(year))
            
            # Ensure local year directory exists
            if not os.path.exists(local_year_dir):
                os.makedirs(local_year_dir)
            
            with sftp.cd(year_dir): 
                files = sftp.listdir()
                for file in files:
                    sftp.get(file, os.path.join(local_year_dir, file))  # Download each file
            
            print(f"Year {year} has been downloaded.")

    print(f"Files from {start_year} to {end_year} have been downloaded.")

Available years: ['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']
Year 2000 has been downloaded.
Year 2001 has been downloaded.
Year 2002 has been downloaded.
Year 2003 has been downloaded.
Year 2004 has been downloaded.
Year 2005 has been downloaded.
Year 2006 has been downloaded.
Year 2007 has been downloaded.
Year 2008 has been downloaded.
Year 2009 has been downloaded.
Year 2010 has been downloaded.
Year 2011 has been downloaded.
Year 2012 has been downloaded.
Year 2013 has been downloaded.
Year 2014 has been downloaded.
Year 2015 has been downloaded.
Year 2016 has been downloaded.
Year 2017 has been downloaded.
Year 2018 has been downloaded.
Year 2019 has been downloaded.
Year 2020 has been downloaded.
Files from 2000 to 2020 have been downloaded.


Now that all of the burn area files have been downloaded, they need to be uncompressed before they can be interacted with. Run the below code to uncompress all files and delete the compressed versions.

In [2]:
import os
import tarfile

# Order the years ascending, and exclude any system files
sorted_year_folders = sorted([folder for folder in os.listdir(burn_area_folder) if folder.isdigit()], key=int)

# Iterate through each subfolder in the directory
for year_folder in sorted_year_folders:
    year_folder_path = os.path.join(burn_area_folder, year_folder)
    
    # Check if it's a directory and matches the year pattern (e.g., 2000, 2001, etc.)
    if os.path.isdir(year_folder_path) and year_folder.isdigit():
        # Iterate through each file in the year subfolder
        for filename in os.listdir(year_folder_path):
            file_path = os.path.join(year_folder_path, filename)
            
            # Check if the file is a .tar.gz file
            if filename.endswith('.tar.gz'):
                # Extract the .tar.gz file
                with tarfile.open(file_path, 'r:gz') as tar:
                    tar.extractall(path=year_folder_path)
                
                # Delete the .tar.gz file after extraction
                os.remove(file_path)
        print(f"Year {year_folder} has been extracted.")
print("All files extracted! Compressed files were deleted.")

Year 2000 has been extracted.
Year 2001 has been extracted.
Year 2002 has been extracted.
Year 2003 has been extracted.
Year 2004 has been extracted.
Year 2005 has been extracted.
Year 2006 has been extracted.
Year 2007 has been extracted.
Year 2008 has been extracted.
Year 2009 has been extracted.
Year 2010 has been extracted.
Year 2011 has been extracted.
Year 2012 has been extracted.
Year 2013 has been extracted.
Year 2014 has been extracted.
Year 2015 has been extracted.
Year 2016 has been extracted.
Year 2017 has been extracted.
Year 2018 has been extracted.
Year 2019 has been extracted.
Year 2020 has been extracted.
All files extracted! Compressed files were deleted.


With the files extracted, we are now ready to interact with them.

As said before, the MCD64 products downloaded here contain the entire contiguous United States, and we are only interested in the state of Montana. Firstly, we will `clip` the burn area files to the state of Montana so that no unnecessary computations are done.

Run the below cell block to clip the burn area shapefiles to the state of Montana:

In [3]:
import geopandas as gpd
import warnings

# Read the state boundary shapefile and determine its CRS
state_shp_folder = "montana_shp"
state_gdf = gpd.read_file(state_shp_folder)
state_crs = state_gdf.crs

# Sometimes shapefiles are empty from no wildfires. Ignore these warnings.
warnings.simplefilter(action='ignore', category=UserWarning)

# Order the years ascending, and exclude any system files
sorted_year_folders = sorted([folder for folder in os.listdir(burn_area_folder) if folder.isdigit()], key=int)

# Iterate through each subfolder in the directory
for year_folder in sorted_year_folders:
    year_folder_path = os.path.join(burn_area_folder, year_folder)
    
    # Check if it's a directory and matches the year pattern (e.g., 2000, 2001, etc.)
    if os.path.isdir(year_folder_path) and year_folder.isdigit():
        # Iterate through each file in the year subfolder
        for filename in os.listdir(year_folder_path):
            file_path = os.path.join(year_folder_path, filename)
            
            # Check if the file is a shapefile (excluding the associated .shx, .dbf, etc. files)
            if filename.endswith('.shp'):
                # Read the shapefile
                gdf = gpd.read_file(file_path)
                
                # Reproject the shapefile to match the Montana shapefile's CRS
                gdf = gdf.to_crs(state_crs)
                
                # Clip the shapefile using the Montana shapefile
                clipped_gdf = gpd.clip(gdf, state_gdf)
                
                # Save the clipped shapefile
                clipped_file_path = os.path.join(year_folder_path, f"clipped_{filename}")
                clipped_gdf.to_file(clipped_file_path)

        print(f"Reprojected and clipped year {year_folder}")

print("All input shapefiles clipped!")

Reprojected and clipped year 2000
Reprojected and clipped year 2001
Reprojected and clipped year 2002
Reprojected and clipped year 2003
Reprojected and clipped year 2004
Reprojected and clipped year 2005
Reprojected and clipped year 2006
Reprojected and clipped year 2007
Reprojected and clipped year 2008
Reprojected and clipped year 2009
Reprojected and clipped year 2010
Reprojected and clipped year 2011
Reprojected and clipped year 2012
Reprojected and clipped year 2013
Reprojected and clipped year 2014
Reprojected and clipped year 2015
Reprojected and clipped year 2016
Reprojected and clipped year 2017
Reprojected and clipped year 2018
Reprojected and clipped year 2019
Reprojected and clipped year 2020
All input shapefiles clipped!


This above code did not modify the original shapefiles, in case they are needed later. They are saved within the same folder structure as prior, just with the prefix `clipped_` to distinguish them from the downloaded files.

Now all of the touching geometries for each "blob" must be dissolved. As each file represents a calendar month (numbered by julian day), we need to dissolve all touching geometries because we can estimate (with slight confidence) that each "blob" is a single fire event. 

If the blob consists of a dozen individual shapes, we do not want to claim the area had twelve fires for the month. Individual, non-touching geometries are not modified.

This process below is more involved than a standard "dissolve" as we do not just want **all** geometries to merge into one. In any instance where there are burn areas separated geographically, this oversight would make the data unusable for counting and accurate area measurements. Instead, we need to make a temporary attribute called `temp` and assign it to all geometries in the shapefile. Then we will find all touching geometries and dissolve just those.

Run the below code to dissolve all touching geometries.

In [4]:
# Order the years ascending, and exclude any system files
sorted_year_folders = sorted([folder for folder in os.listdir(burn_area_folder) if folder.isdigit()], key=int)

# Iterate through each subfolder in the directory
for year_folder in sorted_year_folders:
    year_folder_path = os.path.join(burn_area_folder, year_folder)
    
    # Check if it's a directory and matches the year pattern (e.g., 2000, 2001, etc.)
    if os.path.isdir(year_folder_path) and year_folder.isdigit():
        # Iterate through each file in the year subfolder
        for filename in os.listdir(year_folder_path):
            file_path = os.path.join(year_folder_path, filename)
            
            # Check if the file is a "clipped_" shapefile (excluding the associated .shx, .dbf, etc. files)
            if filename.startswith('clipped_') and filename.endswith('.shp'):
                # Read the shapefile using geopandas
                gdf = gpd.read_file(file_path)
                
                # Create a temporary attribute and set its value to a constant
                gdf['temp'] = 1
                
                # Dissolve using the temporary attribute
                dissolved_gdf = gdf.dissolve(by='temp')

                # Reset the index of the dissolved GeoDataFrame
                dissolved_gdf = dissolved_gdf.reset_index(drop=True)

                
                # Define the output path for the dissolved shapefile
                dissolved_file_path = os.path.join(year_folder_path, filename.replace("clipped_", "dissolved_"))
                
                # Save the dissolved shapefile
                dissolved_gdf.to_file(dissolved_file_path)

        print(f"Year {year_folder} dissolved.")

print("All shapefiles dissolved!")

Year 2000 dissolved.
Year 2001 dissolved.
Year 2002 dissolved.
Year 2003 dissolved.
Year 2004 dissolved.
Year 2005 dissolved.
Year 2006 dissolved.
Year 2007 dissolved.
Year 2008 dissolved.
Year 2009 dissolved.
Year 2010 dissolved.
Year 2011 dissolved.
Year 2012 dissolved.
Year 2013 dissolved.
Year 2014 dissolved.
Year 2015 dissolved.
Year 2016 dissolved.
Year 2017 dissolved.
Year 2018 dissolved.
Year 2019 dissolved.
Year 2020 dissolved.
All shapefiles dissolved!


Now that all the burn area data has been clipped and dissolved, it needs to be merged to form a complete ["fire season"](https://www.fs.usda.gov/fs-tags/fire-season). In conversation with Dr. Giglio, Montana has been described as essentially having the same fire season as the calendar year, so all dissolved shapefiles for the given year will now be combined into one shapefile. This is the last step before insights can be gathered on the data.

Run the below code block to merge all monthly shapefiles into yearly shapefiles:

In [5]:
import pandas as pd
import warnings

# Order the years ascending, and exclude any system files
sorted_year_folders = sorted([folder for folder in os.listdir(burn_area_folder) if folder.isdigit()], key=int)

# Sometimes shapefiles are empty from no wildfires. Ignore these warnings.
warnings.simplefilter(action='ignore', category=UserWarning)

# Iterate through each subfolder in the directory
for year_folder in sorted_year_folders:
    year_folder_path = os.path.join(burn_area_folder, year_folder)
    
    # Check if it's a directory and matches the year pattern (e.g., 2000, 2001, etc.)
    if os.path.isdir(year_folder_path) and year_folder.isdigit():
        # List to store the GeoDataFrames from each "dissolved_" shapefile
        gdf_list = []
        
        # Iterate through each file in the year subfolder
        for filename in os.listdir(year_folder_path):
            file_path = os.path.join(year_folder_path, filename)
            
            # Check if the file is a "dissolved_" shapefile (excluding the associated .shx, .dbf, etc. files)
            if filename.startswith('dissolved_') and filename.endswith('.shp'):
                # Read the shapefile using geopandas
                gdf = gpd.read_file(file_path)
                
                # Append the GeoDataFrame to the list
                gdf_list.append(gdf)
        
        # Merge all the GeoDataFrames in the list
        merged_gdf = pd.concat(gdf_list, ignore_index=True)
        
        # Define the output path for the yearly shapefile
        yearly_file_path = os.path.join(year_folder_path, f"{year_folder}_yearly.shp")
        
        # Save the merged GeoDataFrame as a yearly shapefile
        merged_gdf.to_file(yearly_file_path)

    print(f"Merged year {year_folder}.")

print("Merged all monthly shapefiles into yearly shapefiles.")

Merged year 2000.
Merged year 2001.
Merged year 2002.
Merged year 2003.
Merged year 2004.
Merged year 2005.
Merged year 2006.
Merged year 2007.
Merged year 2008.
Merged year 2009.
Merged year 2010.
Merged year 2011.
Merged year 2012.
Merged year 2013.
Merged year 2014.
Merged year 2015.
Merged year 2016.
Merged year 2017.
Merged year 2018.
Merged year 2019.
Merged year 2020.
Merged all monthly shapefiles into yearly shapefiles.


Now we can derive metrics from the data! The Montana shapefile contains county-level boundaries and their names, so we can use this to determine how many wildfire events each county has each year, the total area of wildfire burns in each county, and the percentage of total land in the county that has burned.

To do this, we first need to convert the Montana shapefile and yearly burn area shapefiles from the degree-based Coordinate Reference System into a meter-based CRS so we can calculate area accurately.

To recreate the data format used in Dr. Silva's regression algorithm, we will store the output as a table, sorted first by the ascending year of the data, then we sort by county name

In [6]:
# Convert the CRS to a local one suitable for area calculations (e.g., UTM Zone 12N for Montana)
local_crs = "EPSG:32612"  # UTM Zone 12N
state_gdf = state_gdf.to_crs(local_crs)

# Create an empty main dataframe to store the results
main_df = pd.DataFrame(columns=["NAME", "YEAR", "COUNT", "AREA_KM^2", "PERCENT"])

# Order the years ascending, and exclude any system files
sorted_year_folders = sorted([folder for folder in os.listdir(burn_area_folder) if folder.isdigit()], key=int)

# Iterate through each subfolder in the directory
for year_folder in sorted_year_folders:
    year_folder_path = os.path.join(burn_area_folder, year_folder)
    
    # Check if it's a directory and matches the year pattern (e.g., 2000, 2001, etc.)
    if os.path.isdir(year_folder_path) and year_folder.isdigit():
        yearly_file_path = os.path.join(year_folder_path, f"{year_folder}_yearly.shp")
        
        if os.path.exists(yearly_file_path):
            # Read the yearly shapefile
            yearly_gdf = gpd.read_file(yearly_file_path).to_crs(local_crs)

            # If the GeoDataFrame is empty, append zeros for that year and continue to the next iteration
            if yearly_gdf.empty:
                year_df = pd.DataFrame({
                    "NAME": state_gdf["NAME"],
                    "YEAR": year_folder,
                    "COUNT": [0] * len(state_gdf),
                    "AREA_KM^2": [0] * len(state_gdf),
                    "PERCENT": [0] * len(state_gdf)
                })
                main_df = pd.concat([main_df, year_df], ignore_index=True)
                continue

            # Perform a spatial join between the yearly shapefile and the Montana shapefile
            joined_gdf = gpd.sjoin(yearly_gdf, state_gdf[['geometry', 'NAME']], how="inner", predicate="intersects")
            
            # Group by the county name and calculate the required metrics
            grouped = joined_gdf.groupby("NAME")
            count = grouped.size()
            area = grouped.geometry.apply(lambda x: x.unary_union.area).divide(1e6)  # Convert from m^2 to km^2
            
            # Get the area of each county
            county_area = state_gdf.set_index("NAME").geometry.area.divide(1e6)  # Convert from m^2 to km^2
            
            # Calculate the percentage of the burned area for each county
            percent = area.divide(county_area).multiply(100)
            
            # Default DataFrame for all counties
            default_data = {
                "NAME": state_gdf["NAME"],
                "YEAR": year_folder,
                "COUNT": [0] * len(state_gdf),
                "AREA_KM^2": [0] * len(state_gdf),
                "PERCENT": [0] * len(state_gdf)
            }
            year_df = pd.DataFrame(default_data)

            # Update the DataFrame with actual values for counties that had wildfires
            for county in count.index:
                year_df.loc[year_df["NAME"] == county, "COUNT"] = count[county]
                year_df.loc[year_df["NAME"] == county, "AREA_KM^2"] = area[county]
                year_df.loc[year_df["NAME"] == county, "PERCENT"] = percent[county]

        # Append this year's results to the main dataframe
        main_df = pd.concat([main_df, year_df], ignore_index=True)

# Sort the main dataframe by year and then by county name
main_df = main_df.sort_values(by=["YEAR", "NAME"]).reset_index(drop=True)

# Display results to user:
main_df

,NAME,YEAR,COUNT,AREA_KM^2,PERCENT
0,Beaverhead,2000,0,0,0
1,Big Horn,2000,0,0,0
2,Blaine,2000,0,0,0
3,Broadwater,2000,0,0,0
4,Carbon,2000,0,0,0
...,...,...,...,...,...
1171,Treasure,2020,1,666.086659,26.419181
1172,Valley,2020,3,784.388145,5.963542
1173,Wheatland,2020,1,67.218629,1.816731
1174,Wibaux,2020,0,0.0,0.0


In [7]:
# Save as CSV
main_df.to_csv('burnarea.csv')